# Import 

In [1]:
# 3d library
import open3d as o3d

In [2]:
# image elaboration libraries

#!pip install Pillow 
#!pip install imageio
#!pip install cv2-plt-imshow
from IPython.display import Image
from imageio import imread
from imageio import imwrite
from PIL import Image
from IPython.display import Image as Im

import cv2
import matplotlib.pyplot as plt
from cv2_plt_imshow import cv2_plt_imshow, plt_format

In [3]:
# others
import numpy as np
import os
import random as myrand

from tqdm.notebook import tqdm# progress bar
import sys
from time import time 
import copy

In [4]:
delimiter = ', ' 
myrand.seed(42)
dash = '-'*80

In [5]:
from datetime import date
from datetime import datetime
import pickle as pkl

# os and directories

In [6]:
from sys import platform
platform

'darwin'

In [7]:
os.chdir('/Users/ariannataormina/Documents/GitHub/02507-Project_work_Image_Analysis_and_Computer_Graphics/')
cwd = os.getcwd()
cwd

'/Users/ariannataormina/Documents/GitHub/02507-Project_work_Image_Analysis_and_Computer_Graphics'

In [8]:
if platform == "linux" or platform == "linux2":
    # linux
    print ("linux to be implemented")
elif platform == "darwin":
    # OS X
    ROOT= cwd +"/data" #mac
    os.chdir(ROOT)
    os.listdir(ROOT)
    #!ls {ROOT}
elif platform == "win32":
    # Windows
    ROOT='ARI_HD:\\' #w10
    os.chdir(ROOT)
    os.listdir(ROOT)
    !dir {ROOT} 

In [9]:
img_folder = ROOT +"/imgfolder"
ply_folder = ROOT +"/plyfolder"

In [10]:
all_images = sorted(os.listdir(img_folder))
all_ply = sorted(os.listdir(ply_folder))

In [11]:
all_paths_images = [img_folder+"/"+all_images[idx] for idx in range(len(all_images))]
all_paths_ply = [ply_folder+"/"+all_ply[idx] for idx in range(len(all_ply))]

In [12]:
img_folder+"/"+all_images[0]

'/Users/ariannataormina/Documents/GitHub/02507-Project_work_Image_Analysis_and_Computer_Graphics/data/imgfolder/image0.png'

## run notebooks

In [13]:
%run "../Notebooks/Stitching_functions.ipynb"
%run "../Notebooks/Visualization_functions.ipynb"
%run "../Notebooks/Downsampling_and_outliers_functions.ipynb"
%load_ext autoreload

## Load the pointclouds

In [19]:
## parameters initialization 

myparams = "./ScreenCamera_2021-01-11-12-06-34.json"  # parameter for camera point view, json file via pressing P
myconfiguration_file = "RenderOption_2021-01-11-12-22-53.json" # configuration file for properties, json file via pressing o


In [22]:
stitched_external_pc = o3d.io.read_point_cloud("stitched_external_pc.pcd")
stitched_internal_pc = o3d.io.read_point_cloud("stitched_internal_pc.pcd")
stitched_upper_pc = o3d.io.read_point_cloud("stitched_upper_pc.pcd")

In [23]:
labels_stitches = ["stitched_external_pc","stitched_internal_pc", "stitched_upper_pc"]
list_stitches = [stitched_external_pc,stitched_internal_pc,stitched_upper_pc]
color_stitches = [[1, 0.706, 0],[0, 0.651, 0.929],[0,0.7,0.3]]

## Plot

In [24]:
##one plot
custom_draw_geometry(stitched_external_pc,
                     mytitle = "stitched_external_pc",
                     params = myparams,  # parameter for camera point view, json file via pressing P
                     configuration_file = myconfiguration_file, # configuration file for properties, json file via pressing o
                     take_screen_shot = False,
                     rotate = True)

loading parameters: 
  ./ScreenCamera_2021-01-11-12-06-34.json
loading configuration file: 
  RenderOption_2021-01-11-12-22-53.json
[Open3D WARNING] GLFW Error: The GLFW library is not initialized


## Many plots

In [25]:
# plot a list of geometries
custom_draw_geometry([list_stitches[:]],
                     mytitle = [labels_stitches[:]],
                     params = myparams,  # parameter for camera point view, json file via pressing P
                     configuration_file = myconfiguration_file, # configuration file for properties, json file via pressing o
                     take_screen_shot = False,
                     rotate = True)

loading parameters: 
  ./ScreenCamera_2021-01-11-12-06-34.json
loading configuration file: 
  RenderOption_2021-01-11-12-22-53.json
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
loading parameters: 
  ./ScreenCamera_2021-01-11-12-06-34.json
loading configuration file: 
  RenderOption_2021-01-11-12-22-53.json
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
loading parameters: 
  ./ScreenCamera_2021-01-11-12-06-34.json
loading configuration file: 
  RenderOption_2021-01-11-12-22-53.json
[Open3D WARNING] GLFW Error: The GLFW library is not initialized


### downsampling

In [27]:
downsampled_stitches = downsample_list_stitches(list_stitches,
                                                #labels_stitches,color_stitches , 
                                                voxel_size = 0.3,
                                                visualization_on = True,
                                                print_statements = True)

Downsampled_pcl_1
loading parameters: 
  ./ScreenCamera_2021-01-11-12-06-34.json
loading configuration file: 
  RenderOption_2021-01-11-12-22-53.json
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
Downsampled_pcl_1
number of points original :  230502
number of points with voxel down sample : 16011

Downsampled_pcl_2
loading parameters: 
  ./ScreenCamera_2021-01-11-12-06-34.json
loading configuration file: 
  RenderOption_2021-01-11-12-22-53.json
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
Downsampled_pcl_2
number of points original :  227230
number of points with voxel down sample : 11908

Downsampled_pcl_3
loading parameters: 
  ./ScreenCamera_2021-01-11-12-06-34.json
loading configuration file: 
  RenderOption_2021-01-11-12-22-53.json
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
Downsampled_pcl_3
number of points original :  219648
number of points with voxel down sample : 13335



### merge the 3 stitched parts

In [33]:
%run "../Notebooks/Stitching_functions.ipynb"
%run "../Notebooks/Visualization_functions.ipynb"
%run "../Notebooks/Downsampling_and_outliers_functions.ipynb"
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
now = datetime.now()
dt_string = now.strftime("%d%m%Y-%Hh%Mm%Ss")
print(dt_string)

11012021-13h13m15s


In [84]:
# prepare dataset
voxel_size  =0.2 # with 0.1 doesnt work!
i = 2
j = 0
source_label = labels_stitches[i][9:-3] # just labels parsing
target_label =labels_stitches[j][9:-3]
source = list_stitches[i]
target = list_stitches[j]
processed_source = downsampled_stitches[i]
processed_target = downsampled_stitches[j]

mytitle= dt_string+"-%s_%s"%(source_label,target_label)

In [85]:
#MAIN
source, target, transformation = stitch_sequences(processed_source,processed_target,
                                                  source_label,target_label,
                                                  mmax_iteration = 10**7,
                                                  mmax_validation = 0.999
                                                 )

allpc_downsampled_upper_external
Load two point clouds and disturb initial pose.
removing outliers
loading parameters: 
  ./ScreenCamera_2021-01-11-12-06-34.json
loading configuration file: 
  RenderOption_2021-01-11-12-22-53.json
[Open3D WARNING] GLFW Error: The GLFW library is not initialized

GLOBAL REGISTRATION: RANSAC registration on downsampled point clouds.
Transformation is:
[[ 9.88539863e-01 -2.62497497e-02  1.48660319e-01 -7.05147822e+00]
 [-7.03713189e-02  7.91094357e-01  6.07632780e-01 -3.27569909e+01]
 [-1.33554548e-01 -6.11130648e-01  7.80181078e-01  1.24820851e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
overlapping points :  48

POINT-TO-PLANE ICP registration is applied on original point
distance threshold 0.400.
Transformation is:
[[  0.98770593  -0.03770636   0.15170768  -7.10777873]
 [ -0.05056124   0.84125229   0.5382733  -30.1740234 ]
 [ -0.14792076  -0.53932626   0.8290034   10.52304358]
 [  0.           0.           0.           1.   

## visualize result

In [79]:
draw_registration_result(source, target, transformation, title = "")

In [81]:
upper_external = save_registration_result(source, target, transformation, 
                                        title= mytitle, 
                                        save_result = False,
                                        visualize_result = True)

In [82]:
get_num_points([upper_external], print_statement = True)



number of points in clouds
(0, 29346)


[29346]

## apply transformation on original 

In [86]:
#use the found transformation with original full point source and target

source = list_stitches[i]
target = list_stitches[j]

draw_registration_result(source, target, transformation, title = "")

In [87]:
#save it
upper_external = save_registration_result(source, target, transformation, 
                                        title= mytitle, 
                                        save_result = True,
                                        visualize_result = False)

custom_draw_geometry(upper_external,
                     mytitle = mytitle,
                     params = myparams,  # parameter for camera point view, json file via pressing P
                     configuration_file = myconfiguration_file, # configuration file for properties, json file via pressing o
                     take_screen_shot = False,
                     rotate = True)

loading parameters: 
  ./ScreenCamera_2021-01-11-12-06-34.json
loading configuration file: 
  RenderOption_2021-01-11-12-22-53.json
[Open3D WARNING] GLFW Error: The GLFW library is not initialized


In [88]:
get_num_points([upper_external], print_statement = True)



number of points in clouds
(0, 450150)


[450150]

## downsample new one for next step

In [64]:
downsampled_upper_external = downsample_list_stitches([upper_external],
                                                ["upper_external"],color_stitches= None , 
                                                voxel_size = 0.3,
                                                visualization_on = True,
                                                print_statements = True)

Downsampled_upper_external
loading parameters: 
  ./ScreenCamera_2021-01-11-12-06-34.json
loading configuration file: 
  RenderOption_2021-01-11-12-22-53.json
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
Downsampled_upper_external
number of points original :  450150
number of points with voxel down sample : 27657



In [90]:
# input 
voxel_size  =0.2
j = 1
source_label = "upper_external"
target_label =labels_stitches[j][9:-3]

processed_source = upper_external
processed_target = downsampled_stitches[j]

mytitle= dt_string+"-%s_%s"%(source_label,target_label)

In [91]:
mytitle

'11012021-13h13m15s-upper_external_internal'

In [92]:
source, target,transformation = tqdm(stitch_sequences(processed_source,processed_target,
                                                                  source_label,target_label,
                                                                  mmax_iteration = 10**7,
                                                                  mmax_validation = 0.999
                                                                 )
                                                )

allpc_downsampled_upper_external_internal
Load two point clouds and disturb initial pose.
removing outliers
loading parameters: 
  ./ScreenCamera_2021-01-11-12-06-34.json
loading configuration file: 
  RenderOption_2021-01-11-12-22-53.json
[Open3D WARNING] GLFW Error: The GLFW library is not initialized

GLOBAL REGISTRATION: RANSAC registration on downsampled point clouds.
Transformation is:
[[-0.24400967  0.95540459 -0.16631704 13.71914808]
 [-0.89544531 -0.15612252  0.41689741 49.52148804]
 [ 0.37233986  0.25065481  0.89360796 30.82876441]
 [ 0.          0.          0.          1.        ]]
overlapping points :  18

POINT-TO-PLANE ICP registration is applied on original point
distance threshold 0.400.
Transformation is:
[[-0.24324384  0.95501895 -0.1696209  13.52030847]
 [-0.90170004 -0.1581965   0.40238155 49.83045709]
 [ 0.35744857  0.250824    0.89962094 31.60967493]
 [ 0.          0.          0.          1.        ]]
overlapping points :  154177


  0%|          | 0/3 [00:00<?, ?it/s]

## visualize result

In [93]:
draw_registration_result(source, target, transformation, title = "")

In [94]:
complete_downsampled = save_registration_result(source, target, transformation, 
                                        title= mytitle, 
                                        save_result = False,
                                        visualize_result = True)

In [95]:
get_num_points([complete_downsampled], print_statement = True)



number of points in clouds
(0, 462058)


[462058]

## apply transformation on original 

In [96]:
#use the found transformation with original full point source and target

source = upper_external
target = list_stitches[j]

draw_registration_result(source, target, transformation, title = "")

In [97]:
mytitle

'11012021-13h13m15s-upper_external_internal'

In [98]:
#save it
complete = save_registration_result(source, target, transformation, 
                                        title= mytitle, 
                                        save_result = True,
                                        visualize_result = False)

custom_draw_geometry(complete,
                     mytitle = mytitle,
                     params = myparams,  # parameter for camera point view, json file via pressing P
                     configuration_file = myconfiguration_file, # configuration file for properties, json file via pressing o
                     take_screen_shot = False,
                     rotate = True)

loading parameters: 
  ./ScreenCamera_2021-01-11-12-06-34.json
loading configuration file: 
  RenderOption_2021-01-11-12-22-53.json
[Open3D WARNING] GLFW Error: The GLFW library is not initialized


In [100]:
get_num_points([complete], print_statement = True)



number of points in clouds
(0, 677380)


[677380]

## Bin

In [65]:

source, target, source_down, target_down, source_fpfh, target_fpfh,trans_init = prepare_dataset(processed_source,
                                                                                             processed_target,
                                                                                             voxel_size=voxel_size, 
                                                                                             #trans_init,
                                                                                             mytitle =mytitle,
                                                                                             print_statements = True
                                                                                              )

Load two point clouds and disturb initial pose.
removing outliers


In [ ]:
#save it
source = upper_external
processed_target = downsampled_stitches[j]

complete = save_registration_result(source, target, transformation, 
                                        title= mytitle, 
                                        save_result = True,
                                        visualize_result = False)

In [ ]:
custom_draw_geometry((target+source),
                 mytitle = mytitle,
                 params = myparams,  # parameter for camera point view, json file via pressing P
                 configuration_file = myconfiguration_file, # configuration file for properties, json file via pressing o
                 take_screen_shot = False,
                 rotate = True)

In [ ]:
complete= o3d.io.read_point_cloud("10012021-22h26m24s-stitch_allpc_downsampled_upper_external_internal.pcd")

In [ ]:
### Radius oulier removal
print("Radius oulier removal")
cl, ind = complete.remove_radius_outlier(nb_points=100, radius=0.8)
custom_draw_geometry_outliers(complete, ind, 
                              mytitle = "Radius_oulier_removal", mytuples = "",
                              params = myparams,  # parameter for camera point view, json file via pressing P
                              configuration_file = myconfiguration_file, # configuration file for properties, json file via pressing o
                              fov_step  = 15,
                              rotate = False)

In [ ]:
mytitle ="Statistical outlier removal"
print (mytitle)
my_nb_neighbors=10000
my_std_ratio=0.1

parameters = (my_nb_neighbors,my_std_ratio)
parameters_labels = ("my_nb_neighbors","my_std_ratio")
mytuples = list(zip(parameters_labels,parameters))

cl, ind = complete.remove_statistical_outlier(nb_neighbors=my_nb_neighbors,
                                                std_ratio=my_std_ratio)
display_inlier_outlier(complete, ind, mytitle, mytuples)

In [ ]:
# GLOBAL 
result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size= voxel_size,
                                            print_statements = True
                                           )

In [ ]:
print("Transformation is:")
print(result_ransac.transformation)
overlapping_points = result_ransac.correspondence_set
print ("overlapping points : " ,len(np.asarray(overlapping_points)))

In [ ]:
draw_registration_result(source, target, 
                     result_ransac.transformation,
                     title = "%global registration"
                    )

In [ ]:
# ICP
result_icp = refine_registration(source, target, 
                                             source_fpfh, target_fpfh,
                                             voxel_size,
                                             mytranformation =result_ransac.transformation,
                                             print_statements = True
                                            )

In [ ]:
print("Transformation is:")
print(result_icp.transformation)
overlapping_points = result_icp.correspondence_set
print ("overlapping points : " ,len(np.asarray(overlapping_points)))

In [ ]:
draw_registration_result(source, target, 
                     result_icp.transformation,
                     title = "icp registration"
                    )

In [ ]:
#save it
upper_external = save_registration_result(source, target, result_icp.transformation, 
                                        mytitle, 
                                        save_result = True,
                                        visualize_result = False)

In [ ]:
custom_draw_geometry(upper_external,
                     mytitle = mytitle,
                     params = myparams,  # parameter for camera point view, json file via pressing P
                     configuration_file = myconfiguration_file, # configuration file for properties, json file via pressing o
                     take_screen_shot = False,
                     rotate = True)